<a href="https://colab.research.google.com/github/JulianNeff/IML2020/blob/master/task4_aurel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive/')
from scipy.fftpack import fft
# Imports
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# Fix random seed
np.random.seed(1)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
'''
Read in data and create datasets
'''

from sklearn.model_selection import train_test_split

df_eeg1_train = pd.read_csv('drive/My Drive/AML/Task4/train_eeg1.csv')
df_eeg2_train = pd.read_csv('drive/My Drive/AML/Task4/train_eeg2.csv')
df_emg_train = pd.read_csv('drive/My Drive/AML/Task4/train_emg.csv')
df_y_train = pd.read_csv('drive/My Drive/AML/Task4/train_labels.csv')
df_eeg1_test = pd.read_csv('drive/My Drive/AML/Task4/test_eeg1.csv')
df_eeg2_test = pd.read_csv('drive/My Drive/AML/Task4/test_eeg2.csv')
df_emg_test = pd.read_csv('drive/My Drive/AML/Task4/test_emg.csv')
# Train and validation data sets
eeg1_train_headers = df_eeg1_train.columns
eeg2_train_headers = df_eeg2_train.columns
emg_train_headers = df_emg_train.columns

eeg1_train_all = df_eeg1_train[eeg1_train_headers[1:]].values
eeg2_train_all = df_eeg2_train[eeg1_train_headers[1:]].values
emg_train_all = df_emg_train[eeg1_train_headers[1:]].values
y_train_all = df_y_train['y'].values

eeg1_train_raw, eeg1_validate_raw, eeg2_train_raw, eeg2_validate_raw, emg_train_raw, emg_validate_raw, y_train, y_validate = train_test_split(
    eeg1_train_all, eeg2_train_all, emg_train_all, y_train_all, test_size=0.1)

# Test data set
#id_test = df_eeg1_test['id']
eeg1_test_headers = df_eeg1_test.columns
eeg2_test_headers = df_eeg2_test.columns
emg_test_headers = df_emg_test.columns
eeg1_test_all = df_eeg1_test[eeg1_test_headers[1:]].values
eeg2_test_all = df_eeg2_test[eeg2_test_headers[1:]].values
emg_test_all = df_emg_test[emg_test_headers[1:]].values

In [0]:
#Feature Extraction: Alpha, Beta, Gamma, ..., Power spectrum
#Sample Frequency = sample rate / amount of samples
#--> 4s segments, 512 samples?
#plt.plot(emg_train_raw, y_train=1)
#plt.plot(eeg1_train_raw, y_train[0])
#plt.plot(eeg1_train_raw, y_train[0])

#plt.figure(figsize=(12,4))
#plt.plot(X_train[100])
#plt.show()

UsageError: Line magic function `%plt.plot` not found.


In [0]:
fft_eeg1_train=fft(eeg1_train_raw)
fft_eeg1_val=fft(eeg1_validate_raw)
fft_eeg2_train=fft(eeg2_train_raw)
fft_eeg2_val=fft(eeg2_validate_raw)
fft_emg_train=fft(emg_train_raw)
fft_emg_val=fft(emg_validate_raw)
#Test
fft_eeg1_test=fft(eeg1_test_all)
fft_eeg2_test=fft(eeg2_test_all)
fft_emg_test=fft(emg_test_all)

In [0]:
# Number of samplepoints
N = 512
# sample spacing
#T = 1.0 / 800.0
#x = np.linspace(0.0, N*T, N)
#xf = np.linspace(0.0, 1.0/(2.0*T), N/2)

#fig, ax = plt.subplots()
#ax.plot(xf, 2.0/N * np.abs(yf[:N//2]))
#plt.show()
#plt.plot(fft_eeg1_train)

In [0]:
'''
Feature extraction
'''

def extract_features(X, y=None):
  n_rows = X.shape[0]
  i_rows = 0
  X_features = []
  for row in X:
    data = row[~np.isnan(row)]
    out = ecg.ecg(signal=data, sampling_rate=300, show=False).as_dict()
    templates = out['templates']
    heart_rate = out['heart_rate']
    
    features = np.nan_to_num([heart_rate_mean, heart_rate_std, 
                              template_std_mean, template_std_std,
                              p_mean, q_mean, r_mean, s_mean, t_mean, 
                              p_std, q_std, r_std, s_std, t_std, 
                              p_arg_mean, q_arg_mean, s_arg_mean, t_arg_mean,
                              p_arg_std, q_arg_std, s_arg_std, t_arg_std,
                              mean_mean])
    X_features.append(features)
    if y is not None:
      print("{} / {} - class {}".format(i_rows, n_rows, y[i_rows]))
    else:
      print("{} / {}".format(i_rows, n_rows))
    i_rows += 1

  # Normalize
  X_features = np.array(X_features)
  m = np.nanmean(X_features, axis=0)
  X_features = X_features - m
  s = np.nanstd(X_features, axis=0)
  X_features = X_features / s
  
  return X_features

X_train = extract_features(X_train_raw, y_train)
X_validate = extract_features(X_validate_raw)
X_test = extract_features(X_test_raw)

NameError: ignored

In [0]:
'''
Pre-process data.
'''

from sklearn.impute import SimpleImputer

# Normalize
# Needs to be done in two steps instead of one because of overflow
m = np.nanmean(X_train, axis=0)
X_train = X_train - m
n = np.nanmean(X_test, axis=0)
X_test  = X_test - n
k = np.nanmean(X_validate, axis=0)
X_validate = X_validate - k
s1 = np.nanstd(X_train, axis=0)
s2 = np.nanstd(X_test, axis=0)
s3 = np.nanstd(X_validate, axis=0)
X_train = X_train / s1
X_test  = X_test / s2
X_validate = X_validate / s3

# Replace NaN
strategy = 'constant'
X_train = SimpleImputer(strategy=strategy).fit_transform(X_train)
X_validate = SimpleImputer(strategy=strategy).fit_transform(X_validate)
X_test = SimpleImputer(strategy=strategy).fit_transform(X_test)

In [0]:
'''
Feature selection
'''

from sklearn.feature_selection import SelectKBest, f_regression

select_k_best = SelectKBest(f_regression, k=600)
select_k_best.fit(X_train, y_train)
X_train = select_k_best.transform(X_train)
X_validate = select_k_best.transform(X_validate)
X_test = select_k_best.transform(X_test)
print(X_train.shape)

In [0]:
'''
Create model and train
'''

from sklearn import svm
from sklearn.linear_model import LogisticRegressionCV
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold


#model = LogisticRegressionCV(cv=5, random_state=0, multi_class='multinomial')
model = svm.SVC(C=1,kernel='rbf', 
                class_weight='balanced',
                probability=True,
                decision_function_shape='ovo')
#model = RFECV(estimator=svc, step=1, cv=StratifiedKFold(5))
model.fit(X_train, y_train)

In [0]:
'''
Evaluation
'''

from sklearn.metrics import balanced_accuracy_score

y_train_predict = model.predict(X_train)
score = balanced_accuracy_score(y_train, y_train_predict)
print('BMAC score of training set:   {}'.format(score))
y_validate_predict = model.predict(X_validate)
score = balanced_accuracy_score(y_validate, y_validate_predict)
print('BMAC score of validation set: {}'.format(score))

In [0]:


'''
Predict on test data and write to file.
'''

y_test = model.predict(X_test)
df_pred = pd.DataFrame({'id': id_test, 'y': y_test})
df_pred.to_csv('submission4.csv', index=False)
!cp submission4.csv drive/My\ Drive/AML/Task1
